1. Extract semantic chunks from your JSON (e.g., abstract, claim, paragraph).

2. Use HuggingFaceEmbeddings or OpenAIEmbeddings.

3. Store structured info as metadata.

4. Use metadata filtering + vector similarity for retrieval.


In [2]:
import sys
import os
from pathlib import Path
import time
import json

# Add source directories to Python path
current_dir = Path.cwd()
project_root = current_dir.parent  # Go up one level from notebooks to project root
src_dir = project_root / "src"

# Add paths
sys.path.append(str(src_dir / "data_pipline"))
sys.path.append(str(src_dir / "EU_XML_data_loader"))


from data_pipline import DataPipeline
import get_raw_data_paths_EPO  
from xml_loader_EPO import process_xml_files_list


print(f"📁 Current directory: {current_dir}")
print(f"📁 Project root: {project_root}")
print(f"📁 Source directory: {src_dir}")
print(f"✅ Python paths configured")

📁 Current directory: /app/notebooks
📁 Project root: /app
📁 Source directory: /app/src
✅ Python paths configured


In [3]:
import json
import glob
import os
from langchain.docstore.document import Document
import sys
# Prepare .json file for embedding
def extract_documents(json_data):
    bibliographic = json_data.get("bibliographic_data", {})
    doc_id = bibliographic.get("doc_id", "UNKNOWN")
    documents = []

    # Common metadata to propagate
    common_meta = {
        "doc_id": doc_id,
        "language": bibliographic.get("language"),
        "country": bibliographic.get("country"),
        "doc_number": bibliographic.get("doc_number"),
        "application_number": bibliographic.get("application_number"),
        "publication_date": bibliographic.get("publication_date"),
        "ipc_classes": bibliographic.get("ipc_classes", []),
    }

    # Title (en preferred)
    title_dict = bibliographic.get("title", {})
    title = title_dict.get("en") or next(iter(title_dict.values()), "")
    if title:
        documents.append(Document(
            page_content=title,
            metadata={**common_meta, "section": "title"}
        ))

    # Abstract
    abstract = bibliographic.get("abstract")
    if abstract:
        documents.append(Document(
            page_content=abstract,
            metadata={**common_meta, "section": "abstract"}
        ))

    # Claims
    for claim in json_data.get("claims", []):
        documents.append(Document(
            page_content=claim["text"],
            metadata={**common_meta, "section": "claim", "claim_number": claim.get("claim_number")}
        ))

    # Main sections
    for section in json_data.get("main_sections", []):
        section_name = section.get("heading_text", "UNKNOWN_SECTION")
        for p in section.get("paragraphs", []):
            documents.append(Document(
                page_content=f"{section_name}\n{p['text']}",
                metadata={**common_meta, "section": section_name, "p_id": p.get("p_id")}
            ))

    return documents


In [8]:
# Test the new JSON loading functionality
sys.path.append(str(src_dir / "data_pipline" / "json_loader"))

# Import the JSON loader functions
from json_loader_epo import get_epo_json_file_paths, get_all_json_file_paths, load_json_documents

print("🔍 Testing JSON File Loading Functions")
print("=" * 50)

🔍 Testing JSON File Loading Functions


In [9]:

def get_chunk_size(text: str, total_tokens: int, base_chunk_size: int = 1000, min_chunk_size: int = 300) -> int:
    # total_tokens = count_tokens(text)

    # If it's small enough, return as one chunk
    if total_tokens <= base_chunk_size:
        return total_tokens

    # Try to divide the tokens into balanced parts
    num_splits = total_tokens // base_chunk_size
    if total_tokens % base_chunk_size != 0:
        num_splits += 1

    # Compute new balanced chunk size
    balanced_chunk_size = total_tokens // num_splits

    # Make sure it doesn't go below a minimum
    balanced_chunk_size = int(max(min_chunk_size, balanced_chunk_size))


    return balanced_chunk_size 

In [23]:
# chunk documents
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_documents(documents):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

    processed_docs = []
    for doc in documents:
        token_count =len(str(doc))
        balanced_chunk_size = get_chunk_size(doc, token_count)
        print("Token Count: ", token_count)
        print("balanced_chunk_size: ", balanced_chunk_size)
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=balanced_chunk_size,
            # chunk_size=500,
            chunk_overlap=50
            )

        chunks = splitter.split_documents([doc])
        # print("Chunks: ", chunks)
        print(len(processed_docs))
        processed_docs.extend(chunks)
        print(len(processed_docs))
        # processed_docs.append(doc)
    return processed_docs


In [24]:

import os
os.environ["TRANSFORMERS_HTTP_TIMEOUT"] = "60"
# load the .json documents
file_list = get_epo_json_file_paths()[:100]
# file_list = glob.glob(json_files_path)

all_documents = []

for file_path in file_list:
    with open(file_path, "r") as f:
        data = json.load(f)
        docs = extract_documents(data)
        chunked_docs = chunk_documents(docs)
        all_documents.extend(chunked_docs)



📁 Found 1286 EPO JSON files
Token Count:  330
balanced_chunk_size:  330
0
1
Token Count:  4122
balanced_chunk_size:  374
1
13
Token Count:  523
balanced_chunk_size:  261
13
14
Token Count:  657
balanced_chunk_size:  328
14
15
Token Count:  803
balanced_chunk_size:  267
15
17
Token Count:  4726
balanced_chunk_size:  393
17
30
Token Count:  1025
balanced_chunk_size:  341
30
33
Token Count:  703
balanced_chunk_size:  351
33
35
Token Count:  802
balanced_chunk_size:  267
35
37
Token Count:  489
balanced_chunk_size:  244
37
38
Token Count:  528
balanced_chunk_size:  264
38
39
Token Count:  878
balanced_chunk_size:  292
39
42
Token Count:  536
balanced_chunk_size:  268
42
43
Token Count:  837
balanced_chunk_size:  279
43
46
Token Count:  490
balanced_chunk_size:  245
46
47
Token Count:  514
balanced_chunk_size:  257
47
48
Token Count:  722
balanced_chunk_size:  361
48
51
Token Count:  471
balanced_chunk_size:  235
51
52
Token Count:  588
balanced_chunk_size:  294
52
53
Token Count:  674
bala

In [ ]:
for i, doc in enumerate(all_documents, start=1):
    if i > 2:
        break
    print(f"Document {i}")
    print("Page Content:", doc.page_content)
    print("Metadata: ", doc.metadata)
print("Wtf")